In [1]:
# Import necessary libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time

from splinter import Browser
from selenium.webdriver.firefox.service import Service

my_service = Service(executable_path='/Users/nicolorenzi/Documents/Personal Projects/Flip Finder/geckodriver')
browser = Browser('firefox', service=my_service)

In [14]:
# Set up Splinter
browser = Browser()
# Set up base url
base_url = "https://www.facebook.com/marketplace/toronto/search?"
# Set up search parameters
min_price = 1000
max_price = 30000
days_listed = 7
min_mileage = 50000
max_mileage = 200000
min_year = 2000
max_year = 2020
transmission = "automatic"
make = "Honda"
model = "Civic"
#Set up full url
url = f"{base_url}minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&maxMileage={max_mileage}&maxYear={max_year}&minMileage={min_mileage}&minYear={min_year}&transmissionType={transmission}&query={make}{model}&exact=false"

In [15]:
# Visit the website
browser.visit(url)

In [4]:
if browser.is_element_present_by_css('div[aria-label="Close"]', wait_time=10):
    # Click on the element once it's found
    browser.find_by_css('div[aria-label="Close"]').first.click()

In [5]:
# Scroll down to load more results

# Define the number of times to scroll the page
scroll_count = 4

# Define the delay (in seconds) between each scroll
scroll_delay = 2

# Loop to perform scrolling
for _ in range(scroll_count):
    # Execute JavaScript to scroll to the bottom of the page
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Pause for a moment to allow the content to load
    time.sleep(scroll_delay)

In [6]:
# Parse the HTML
html = browser.html

# Create a BeautifulSoup object from the scraped HTML
market_soup = soup(html, 'html.parser')
# Check if HTML was scraped correctly
market_soup

<html class="_9dls __fb-light-mode" dir="ltr" id="facebook" lang="en"><head><link data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/UJj0tgk-RrT.ico" data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yT/r/aGT3gskzWBf.ico" href="https://static.xx.fbcdn.net/rsrc.php/yT/r/aGT3gskzWBf.ico" rel="shortcut icon"/><meta content="noarchive" name="bingbot"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><link crossorigin="use-credentials" href="/data/manifest/" id="MANIFEST_LINK" rel="manifest"/><meta content="light" name="color-scheme"/><meta content="#FFFFFF" name="theme-color"/><style nonce="">:root{--fds-black:#000000;--fds-black-alpha-05:rgba(0, 0, 0, 0.05);--fds-black-alpha-10:rgba(0, 0, 0, 0.1);--fds-black-alpha-15:rgba(0, 0, 0, 0.15);--fds-black-alpha-20:rgba(0, 0, 0, 0.2);--fds-black-alpha-30:rgba(0, 0, 0, 0.3);--fds-black-alpha-40:rgba(0, 0, 0, 0.4);--fds-black-alpha-50:rgba(0, 0, 0, 0.5);--fds-black-alpha-60:rgba(0, 

In [7]:
# End the automated browsing session
browser.quit()

In [8]:
# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
mileage_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
mileage_list = [mileage.text.strip() for mileage in mileage_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1sur9pj xkrqix3 x1lku1pv")
urls_list = [url.get('href') for url in urls_div]

In [9]:
# Create a regular expression pattern to match city and state entries like "City, State"
pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})')

# Initialize an empty list to store adjusted mileage entries
mileage_list2 = []

# Iterate through the original mileage entries
for item in mileage_list:
    # Append the current mileage entry to the adjusted list
    mileage_list2.append(item)
    
    # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
    if pattern.match(item) and len(mileage_list2) >= 2 and pattern.match(mileage_list2[-2]):
        # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
        mileage_list2.insert(-1, '0K km')

In [10]:
mileage_list2

[]

In [11]:
# Extracted mileage list (separate from location and extract numeric values only)
# Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
mileage_pattern_km = r'(\d+)K km'
mileage_pattern_miles = r'(\d+)K miles'

# Initialize an empty list to store cleaned mileage values
mileage_clean = []

# Iterate through the adjusted mileage entries
for item in mileage_list2:
    # Try to find a match for the "K km" format
    match_mileage_km = re.search(mileage_pattern_km, item)
    
    # Try to find a match for the "K miles" format
    match_mileage_miles = re.search(mileage_pattern_miles, item)
    
    # Check if either of the formats is found
    if match_mileage_km or match_mileage_miles:
        # If "K km" format is found, convert it to meters and append to the cleaned list
        if match_mileage_km:
            mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
        # If "K miles" format is found, convert it to meters and append to the cleaned list
        else:
            mileage_clean.append(int(match_mileage_miles.group(1)) * 1600)

In [12]:
mileage_clean

[]

In [13]:
# Add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    
    title_split = titles_list[i].split()
    
    cars_dict["Year"] = int(title_split[0])
    cars_dict["Make"] = title_split[1]
    cars_dict["Model"] = title_split[2]
    cars_dict["Price"] = int(re.sub(r'[^\d.]', '', prices_list[i]))
    cars_dict["Mileage"] = mileage_clean[i]
    cars_dict["URL"] = urls_list[i]
    vehicles_list.append(cars_dict)
    
print(vehicles_list)

ValueError: invalid literal for int() with base 10: 'Filters'

In [ ]:
vehicles_df = pd.DataFrame(vehicles_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

vehicles_df.tail()

In [ ]:
# Add the prefix to the URLs
vehicles_df['URL'] = 'https://www.facebook.com/' + vehicles_df['URL']

In [ ]:
vehicles_df.sample(10)

In [ ]:
# Filter the DataFrame to include rows where the 'Model' column matches the specified model, regardless of case.
filtered_df = vehicles_df[vehicles_df['Model'].str.lower() == model.lower()]

In [ ]:
filtered_df

In [ ]:
# Group the data by "Year" and calculate the mean price for each year
yearly_prices = vehicles_df.groupby("Year")["Price"].mean()

# Create a wider figure
plt.figure(figsize=(12, 6))

# Create the column chart
plt.bar(yearly_prices.index, yearly_prices.values)

# Add values on top of the bars with no digits after the period
for x, y in zip(yearly_prices.index, yearly_prices.values):
    y = int(y)  # Convert to an integer to remove digits after the period
    plt.text(x, y, f'{y}', ha='center', va='bottom')

# Set the labels and title
plt.xlabel("Year")
plt.ylabel("Price")
plt.title("Average Price by Year")

# Use adaptive scale for the year axis
plt.xticks(yearly_prices.index, rotation=45)

# Show the chart
plt.show()

In [ ]:
# Filter out rows where mileage is zero
filtered_df = filtered_df[filtered_df["Mileage"] != 0]

# Group the data by "Year" and calculate the mean mileage for each year
yearly_miles = filtered_df.groupby("Year")["Mileage"].mean()

# Create a wider figure
plt.figure(figsize=(12, 6))

# Create the column chart
plt.bar(yearly_miles.index, yearly_miles.values)

# Add values on top of the bars
for x, y in zip(yearly_miles.index, yearly_miles.values):
    y = int(y)  # Convert to an integer to remove digits after the period
    plt.text(x, y, f'{y:.0f}', ha='center', va='bottom')

# Set the labels and title
plt.xlabel("Year")
plt.ylabel("Mileage")
plt.title("Average Mileage by Year")

# Use adaptive scale for the year axis
plt.xticks(yearly_miles.index, rotation=45)

# Show the chart
plt.show()

In [ ]:
# Group the data by "Year" and count the number of listings for each year
listings_by_year = filtered_df.groupby("Year").size()

# Create a wider figure
plt.figure(figsize=(12, 6))

# Create a bar chart
plt.bar(listings_by_year.index, listings_by_year.values)

# Add values on top of the bars
for x, y in zip(listings_by_year.index, listings_by_year.values):
    plt.text(x, y, f'{y}', ha='center', va='bottom')

# Set the labels and title
plt.xlabel("Year")
plt.ylabel("Number of Listings")
plt.title("Number of Listings by Year")

# Use adaptive scale for the year axis
plt.xticks(listings_by_year.index, rotation=45)

# Show the chart
plt.show()

In [ ]:
# Find the URL of the cheapest Honda Civic directly from the filtered DataFrame
cheapest_civic_url = filtered_df[filtered_df["Price"] == filtered_df["Price"].min()]["URL"].values[0]

# Print the URL
print(f"{cheapest_civic_url}")

In [ ]:
# Export to CSV

csv_file_path = r'C:\Users\shake\Desktop\Projects\Flip-Finder\cars.csv'

filtered_df.to_csv(csv_file_path, index=False)